# The SVM Classifier

In this notebook we'll run through an implementation of the 
[SVM classifier](https://www.robots.ox.ac.uk/~az/lectures/ml/lect2.pdf) 
in numpy.

In the next notebook, we will implement it in pytorch instead and
showcase the simplicity of working with it over numpy.

Most of this code is taken from 
[my other repository](https://github.com/jmsalvador2395/svm/tree/master)
which shows a lot more visualizations of the weights and 
training results so if you're interested you can go there for more info.

# Initial Setup

In [1]:
import numpy as np
import datasets

from matplotlib import pyplot as plt
from numpy.linalg import norm
from pathlib import Path
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Load in the Data

To do this we'll use the `datasets` library from huggingface.
Don't worry about this for now since this will be covered in later
tutorials.

In [2]:
def get_dataloaders(
		ds_name='mnist', 
		data_dir='./datasets', 
		batch_size=16, 
		get_mean=False):
	"""
	downloads the dataset and returns a dataloader for batching

	:param ds_name: name of the dataset. check huggingface for valid 
					dataset names
	:param data_dir: the directory to download and store the data
	:param batch_size: the batch size for the classifier to train on

	:return: two pytorch dataloaders for the training and testing data
	"""
	# create path for dataset if it doesn't exist
	Path(data_dir).mkdir(parents=True, exist_ok=True)
	
	# download or read in dataset
	ds = datasets.load_dataset(
            ds_name,
            cache_dir = data_dir,
            keep_in_memory=True).with_format('numpy')

	# zero mean procedure. only computes mean based of training set
	if get_mean:
		new_ds = {
			split : {'image' : [], 'label' : []} 
			for split in ds.keys()
		}
		mean_x = np.mean(np.array(
			[sample['image'] for sample in ds['train']]
		), dtype=np.float32)

		return  (
			ds,
			mean_x,
			DataLoader(ds['train'], batch_size=batch_size),
			DataLoader(ds['test'], batch_size=batch_size),
		)
	return  (
		ds,
		DataLoader(ds['train'], batch_size=batch_size),
		DataLoader(ds['test'], batch_size=batch_size),
	)

ds, train_dl, test_dl = get_dataloaders()

# Define the SVM class

Here we define the architecture, loss and gradient calculation, and
optimization steps for the classifier.

We train our model on the loss function
$$
\mathcal{L_\theta(x)}=\lambda R_2(\theta)+\frac{1}{N}\sum\limits_{i=1}^N \max(0, 1-y_if_\theta(x_i))
$$

In [3]:
class SVM:
    """ implementation of the SVM classifier """

    def __init__(
            self, 
            C=10, dim=784, shape=(28, 28), 
            reg=1, lr=1, loc=0, scale=1, 
            decay=1, mean_x=None):
        """
        initialize the weights

        :param C: number of classes
        :param dim: dimensionality of the input
        :param reg: the regularization term
        :param loc: arguement for initialization using 
                    numpy.random.normal()
        :param scale: standard deviation for initialization using 
                      numpy.random.normal()
        """


        # save the learning rate
        self.lr=lr
        self.lr_decayed=lr

        # save the regularization term
        self.reg=reg

        # save dimensionality for reshaping input
        self.dim=dim

        # save the shape of the data
        self.shape=shape

        # save the learning rate decay
        self.decay=decay

        # initialize weights. use +1 for the bias weights
        self.w=np.random.normal(loc=loc, scale=scale, size=(dim+1, C))

        # set bias weights to 0
        self.w[-1, :]=0

        # sets the mean of the data
        self.mean_x = mean_x
    
    def __call__(self, x):
        """
        generates classifier scores. computes x@w+b

        :param x: the input data

        :return: prediction scores
        """

        # get number of samples
        N=x.shape[0]
        if self.mean_x:
            N-=self.mean_x

        # reshape x to an Nxdim matrix and then pad with 1s
        xpad=np.hstack((
            x.reshape(N, self.dim),
            np.ones(N)[:, None]
        ))

        return xpad@self.w

    def loss(self, x, y=None):
        """
        this is used for calculating the loss function
        
        :param x: the input data
        :param y: the labels for the input data

        :return loss: the output of the loss function
        :return grad: the weight gradient
        :return acc: the accuracy of the model
        """

        # get the number of samples
        N=x.shape[0]

        # reshape x to an Nxdim matrix and then pad with 1s
        xpad=np.hstack((
            x.reshape(N, self.dim),
            np.ones(N)[:, None]
        ))

        # use this to index the scores
        xi=range(N)

        # calculate scores
        scores=self(x)

        preds=np.argmax(scores, axis=-1)

        # compute accuracy
        acc=np.mean(preds==y)
        #acc=np.sum(preds==y)

        # compute loss
        ys=-np.ones(scores.shape)
        ys[xi, y]=1
        svm_scores=np.maximum(0, 1-ys*scores)
        loss=np.mean(svm_scores)+self.reg*norm(self.w, ord=2)

        # compute gradient
        mask=(svm_scores==0)
        grad=-ys
        grad[mask]=0
        grad=xpad.T@grad/grad.size+self.reg*2*self.w

        return loss, grad, acc
    
    def optim_step(self, grad):
        """
        use this for the optimization step. Implementation is SGD

        :param grad: the gradient of the current training step
        """
        self.w-=self.lr_decayed*grad
        self.lr_decayed = max(self.lr_decayed*self.decay, 1e-3)

    def w_norm(self, ord=2):
        """return the sum of norms of the weight vectors"""
        return np.linalg.norm(self.w, ord=2)

# Define the Training Procedure

In [6]:
def evaluate(model, data):
    N=len(data)
    loss=0
    acc=0
    # compute loss and accuracy on the test set
    for batch in data:

        # split into data and labels. 
        x, y = (batch['image'].numpy(), batch['label'].numpy())

        # compute loss
        loss_i, _, acc_i = model.loss(x, y)

        loss+=loss_i
        acc+=acc_i
    
    return loss/N, acc/N

def train(
        model=None, C=10, 
        dim=784, lr=.01, 
        decay=.995, reg=1, 
        batch_size=4096, 
        num_epochs=15, mean_x=None):
    """ instantiate and train an SVM classifier """

    # instantiate model
    model = SVM(
        C,
        dim,
        lr=lr,
        decay=decay,
        reg=reg,
        mean_x=mean_x,
    )

    # get dataset
    _, train_data, test_data = get_dataloaders(batch_size=batch_size)

    history={
        'train_loss':[],
        'train_acc':[],
        'test_loss':[],
        'test_acc':[],
        'w':[],
    }

    bar=tqdm(range(num_epochs*len(train_data)))
    import time
    
    # outer training loop is for each epoch
    for epoch in range(num_epochs):
        
        # inner loop iterates through the whole dataset
        for batch in train_data:
            # update progress bar
            bar.update()

            # split into data and labels. 
            data, labels = (
                batch['image'].detach().clone().numpy(), 
                batch['label'].detach().clone().numpy()
            )
                
            # compute loss
            loss, grad, acc = model.loss(data, labels)

            # collect historical data
            history['train_loss'].append(loss)
            history['train_acc'].append(acc)

            # optimizer step
            model.optim_step(grad)

            # collect historical data
            test_loss, test_acc = evaluate(model, test_data)
            history['test_loss'].append(test_loss)
            history['test_acc'].append(test_acc)
            history['w'].append(model.w.copy())
            
            # update progress bar
            bar.set_postfix({
                'loss_trn':history['train_loss'][-1],
                'loss_tst':history['test_loss'][-1],
                'acc_trn':history['train_acc'][-1],
                'acc_tst':history['test_acc'][-1],
                'lr':model.lr_decayed,
                '||w||':np.linalg.norm(history['w'][-1], ord=2),
            })
            


    return model, history

# Train the Model

In [7]:
model, history = train(
    lr=.01,
    decay=.995,
    reg=1,
    num_epochs=40,
)

  0%|          | 0/600 [00:00<?, ?it/s]

forward took 0.03 seconds
forward took 0.04 seconds
forward took 0.03 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.03 seconds
forward took 0.03 seconds
forward took 0.04 seconds
forward took 0.03 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.03 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.02 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took 0.04 seconds
forward took